In [1]:
save_path = r"\depth_mode\Savings\jp-rinna-japanese-roberta-base_bs1_seed111\\"
data_path = r"\data\pickle-data\depth\to_pt\jp-rinna-japanese-roberta-base"

In [2]:
import math
import torch.optim as optim
import random
import torch.nn as nn
import copy
import os
import glob
import Metric
from Metric import getBatchMeasure, getMicroMeasure, getMacroMeasure

In [3]:
import re
import os
import pickle
import torch
import numpy as np
import random
from Training import Train, EvalOnly, getAccuracy
import time
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel, MBart50Tokenizer
import config
from model_depth import ParsingNet

os.environ["CUDA_VISIBLE_DEVICES"] = str(config.global_gpu_id)
base_path = config.tree_infer_mode + "_mode/"


USE_CUDA = torch.cuda.is_available() #False
device = torch.device("cuda:" + str(config.global_gpu_id) if USE_CUDA else "cpu") #device(type='cpu')

In [4]:
batch_size = 1
hidden_size = config.hidden_size
rnn_layers = 1
dropout_e = 0.5
dropout_d = 0.5
dropout_c = 0.5
input_is_word = 'True'
atten_model = 'Dotproduct'
classifier_input_size = config.hidden_size #768
classifier_hidden_size = int(config.hidden_size / 1) #768
classifier_bias = 'True'
use_org_Parseval = 'True'
eval_only = 'True'

seednumber = 111
data_path_split = [x for x in data_path.split(os.sep) if x != ""]
data_base = data_path_split[-1] #'zh-gcdt-hfl-chinese-roberta-wwm-ext'
savepath = r"Savings\jp-rinna-japanese-roberta-base_bs1_seed111\\"
finetuning = "False"
if savepath == None and finetuning == "True":
    assert False
elif savepath == None and finetuning == "False":
    save_path = base_path + "Savings/%s_bs%d_seed%d/" % (data_base, batch_size, seednumber)
    finetune_frompath = None
elif savepath != None and finetuning == "True":
    finetune_frompath = base_path + savepath
    save_path = finetune_frompath.replace("Savings/", "Savings/finetuned_")
elif savepath != None and finetuning == "False":
    finetune_frompath = None
    save_path = base_path + savepath
        
eval_size = 1
epoch = 50
lr = 0.0002
lr_decay_epoch = 1
weight_decay = 0.01

In [5]:
# get language embedding
language_embedding_name = re.sub(r"jp-", "", re.sub(r".*gcdt-", "", (re.sub(r".*rstdt-", "", re.sub(r".*gum-", "", data_base)))))
language_embedding_name = language_embedding_name.replace("rinna-", "rinna/")
language_embedding_name = language_embedding_name.replace("hfl-", "hfl/").replace("SpanBERT-", "SpanBERT/")
# 'hfl/chinese-roberta-wwm-ext' a model on Hugging Face

""" BERT tokenizer and model """
print("language embedding name: ", language_embedding_name)
bert_tokenizer = AutoTokenizer.from_pretrained(language_embedding_name, use_fast=True)
bert_model = AutoModel.from_pretrained(language_embedding_name)
print("Language embedding loaded from pretrained: ", language_embedding_name)

""" freeze some layers """
for name, param in bert_model.named_parameters():
    layer_num = re.findall("layer\.(\d+)\.", name)
    if len(layer_num) > 0 and int(layer_num[0]) > 2:
        param.requires_grad = True
    else:
        param.requires_grad = False

if USE_CUDA:
    language_model = bert_model.cuda()
else:
    language_model = bert_model

# Setting random seeds 
torch.manual_seed(seednumber)
if USE_CUDA:
    torch.cuda.manual_seed_all(seednumber)
np.random.seed(seednumber)
random.seed(seednumber)

# Process bool args       
if classifier_bias == 'True':
    classifier_bias = True
    # True

elif classifier_bias == 'False':
    classifier_bias = False

language embedding name:  rinna/japanese-roberta-base


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Some weights of RobertaModel were not initialized from the model checkpoint at rinna/japanese-roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Language embedding loaded from pretrained:  rinna/japanese-roberta-base


In [6]:
Tr_InputSentences = []
Tr_EDUBreaks = []
Tr_DecoderInput = []
Tr_RelationLabel = []
Tr_ParsingBreaks = []
Tr_GoldenMetric = []
Tr_ParentsIndex = []
Tr_SiblingIndex = []

Dev_InputSentences = []
Dev_EDUBreaks = []
Dev_DecoderInput = []
Dev_RelationLabel = []
Dev_ParsingBreaks = []
Dev_GoldenMetric = []
Dev_ParentsIndex = []
Dev_SiblingIndex = []

# Load Testing data
Test_InputSentences = []
Test_EDUBreaks = []
Test_DecoderInput = []
Test_RelationLabel = []
Test_ParsingBreaks = []
Test_GoldenMetric = []

# Load Training data
Tr_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Training_InputSentences.pickle"), "rb")))
Tr_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Training_EDUBreaks.pickle"), "rb")))
Tr_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Training_DecoderInputs.pickle"), "rb")))
Tr_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Training_RelationLabel.pickle"), "rb")))
Tr_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Training_ParsingIndex.pickle"), "rb")))
Tr_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Training_GoldenLabelforMetric.pickle"), "rb")))
Tr_ParentsIndex.extend(pickle.load(open(os.path.join(data_path, "Training_ParentsIndex.pickle"), "rb")))
Tr_SiblingIndex.extend(pickle.load(open(os.path.join(data_path, "Training_Sibling.pickle"), "rb")))

# Load Deving data
Dev_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Deving_InputSentences.pickle"), "rb")))
Dev_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Deving_EDUBreaks.pickle"), "rb")))
Dev_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Deving_DecoderInputs.pickle"), "rb")))
Dev_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Deving_RelationLabel.pickle"), "rb")))
Dev_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Deving_ParsingIndex.pickle"), "rb")))
Dev_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Deving_GoldenLabelforMetric.pickle"), "rb")))
Dev_ParentsIndex.extend(pickle.load(open(os.path.join(data_path, "Deving_ParentsIndex.pickle"), "rb")))
Dev_SiblingIndex.extend(pickle.load(open(os.path.join(data_path, "Deving_Sibling.pickle"), "rb")))

# Load Testing data
Test_InputSentences.extend(pickle.load(open(os.path.join(data_path, "Testing_InputSentences.pickle"), "rb")))
Test_EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "Testing_EDUBreaks.pickle"), "rb")))
Test_DecoderInput.extend(pickle.load(open(os.path.join(data_path, "Testing_DecoderInputs.pickle"), "rb")))
Test_RelationLabel.extend(pickle.load(open(os.path.join(data_path, "Testing_RelationLabel.pickle"), "rb")))
Test_ParsingBreaks.extend(pickle.load(open(os.path.join(data_path, "Testing_ParsingIndex.pickle"), "rb")))
Test_GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "Testing_GoldenLabelforMetric.pickle"), "rb")))

In [7]:
# To check data
sent_temp = ''
print("Checking Data...")
for word_temp in Tr_InputSentences[2]:
    sent_temp = sent_temp + ' ' + word_temp
print(sent_temp)
print('... ...')
print("That's great! No error found!")
print("All train sample number:", len(Tr_InputSentences))

Checking Data...
 ▁ 加藤 ▁ 周 一 ▁ の ▁「 ▁ 眼 ▁ 」 ▁ と ▁「 ▁ 耳 ▁ 」 ▁ 加 國 ▁尚 志 ▁ メル ロ ▁= ▁ ポン ティ ▁ 晩年 ▁ の ▁ 哲学 ▁ 概念 ▁ に ▁「 ▁ 肉 ▁la ▁cha ir ▁ 」 ▁ と ▁ いう ▁ 概念 ▁ が ▁ある ▁ 。 ▁これ ▁ は ▁たとえば ▁ 右手 ▁ と ▁ 左手 ▁ を ▁ 重ね 合わせる ▁ とき ▁ 、 ▁ 触れ る ▁ 手 ▁ と ▁ 触れ ▁ られる ▁ 手 ▁ の ▁ 間 ▁ に ▁ 可逆 ▁ 的 ▁ で ▁ あい まい ▁ な ▁ 感覚 ▁ が ▁ 生じ ▁ 、 ▁ 主 観 ▁ と ▁ 客 観 ▁ 、 ▁ 能 動 ▁ と ▁ 受 動 ▁ が ▁ 相互 ▁ に ▁ 移り ▁ 行き あう ▁ よう ▁ な ▁「 ▁ 感じ ▁ られる ▁ もの ▁le ▁se ns ible ▁ 」 ▁ を ▁ 言い 表 し ▁ た ▁ もの ▁ で ▁ある ▁ 。 ▁この ▁「 ▁ 感じ ▁ られる ▁ もの ▁ 」 ▁ は ▁「 ▁ 見える ▁ もの ▁ 」 ▁ や ▁「 ▁ 触れ ▁ うる ▁ もの ▁ 」 ▁ に ▁ 宿 り ▁ 、 ▁さらに ▁ は ▁それ ▁ ら ▁ 相互 ▁ の ▁ 転移 ▁ を ▁ 可能 ▁ に ▁ する ▁ 。 ▁この ▁「 ▁ 肉 ▁ 」 ▁ の ▁ 経験 ▁ が ▁たとえば ▁ 色 ▁ など ▁ の ▁ 質 ▁ の ▁ 経験 ▁ の ▁ 基礎 ▁ に ▁ あり ▁ 、 ▁ 感 性 ▁ 的 ▁ な ▁ 現象 ▁ を ▁ 現象 ▁ と ▁ し ▁ て ▁ 成立 ▁ さ ▁ せる ▁ 存在 論 ▁ 的 ▁ な ▁ 概念 ▁ と ▁ し ▁ て ▁ 主 客 ▁二 ▁元 ▁ 論 ▁ の ▁ 近代 ▁ 哲学 的 ▁ 図 式 ▁ を ▁ 解体 ▁ さ ▁ せる ▁ もの ▁ と ▁ なる ▁ こと ▁ を ▁ メル ロ ▁= ▁ ポン ティ ▁ は ▁ 目 論 ん ▁ で ▁ い ▁ た ▁ 。 ▁『 ▁ 眼 ▁ と ▁ 精神 ▁ 』 ▁( ▁1961 ▁ 年 ▁ ) ▁ は ▁この ▁「 ▁ 肉 ▁ 」 ▁ の ▁ 概念 ▁ を ▁ 絵画 ▁ や ▁ 彫刻 ▁ など ▁ の ▁ 美術 ▁ 作品 ▁ に ▁ 適用 ▁ しよう ▁ と ▁ し 

In [8]:
# To save model and data
FileName = str(seednumber) + "_" + config.tree_infer_mode + '_Batch_' + str(batch_size) + 'Hidden_' + str(hidden_size) + \
            'LR' + str(lr) + "_" + str(time.time())

if not os.path.isdir(save_path):
    os.makedirs(save_path)
print("Model save path", save_path)
print("Finetune from path", finetune_frompath)
        

""" relation number is set at 42 """
if "en-rstdt" in data_path:
    number_of_relations = 42
elif "-gum" in data_path or "-gcdt" in data_path or "jp" in data_path:
    number_of_relations = 30
model = ParsingNet(language_model, hidden_size, hidden_size,
                    hidden_size, atten_model, classifier_input_size, classifier_hidden_size, number_of_relations,
                    classifier_bias, rnn_layers, dropout_e, dropout_d, dropout_c, bert_tokenizer=bert_tokenizer)

if USE_CUDA:
    model = model.cuda()


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

    

Model save path depth_mode/Savings\jp-rinna-japanese-roberta-base_bs1_seed111\\
Finetune from path None


In [9]:
with torch.no_grad():  # reduce memory
        # Load model
        torchsave_files = sorted(glob.glob(save_path + "Epoch_*.torchsave"))
        assert len(torchsave_files) == 1
        torchsave_best = torchsave_files[0]
        best_epoch_Dev = int(re.search(r"\d+", os.path.basename(torchsave_best)).group())
            
        model.load_state_dict(torch.load(torchsave_best, map_location=device))

        # Convert model to eval
        model.eval()

        # Eval on Testing data
        LossTree_Test, LossLabel_Test, span_points_Test, relation_points_Test, \
        nuclearity_points_Test, F1_full_Test, segment_points_Test = getAccuracy(Test_InputSentences,
                                                                                Test_EDUBreaks,
                                                                                Test_DecoderInput,
                                                                                Test_RelationLabel,
                                                                                Test_ParsingBreaks,
                                                                                Test_GoldenMetric,
                                                                                use_pred_segmentation=False,
                                                                                use_org_Parseval=use_org_Parseval,
                                                                                batch_size=eval_size,
                                                                                model=model)
        
        # Unfold numbers
        # Test
        P_span_Test, R_span_Test, F_span_Test = span_points_Test
        P_relation_Test, R_relation_Test, F_relation_Test = relation_points_Test
        P_nuclearity_Test, R_nuclearity_Test, F_nuclearity_Test = nuclearity_points_Test
        P_segment_Test, R_segment_Test, F_segment_Test = segment_points_Test
        
        print('Epoch EvalOnly Test:\t%d\n' % best_epoch_Dev,
              'F_segment_Test\tF_span_Test\tF_nuclearity_Test\tF_relation_Test:\n%.4f\t%.4f\t%.4f\t%.4f'
              % (F_segment_Test, F_span_Test, F_nuclearity_Test, F_relation_Test))
        
        # Eval on each document
        for docid in range(len(Test_InputSentences)):
            LossTree_Test, LossLabel_Test, span_points_Test, relation_points_Test, \
            nuclearity_points_Test, F1_full_Test, segment_points_Test = getAccuracy([Test_InputSentences[docid]],
                                                                                    [Test_EDUBreaks[docid]],
                                                                                    [Test_DecoderInput[docid]],
                                                                                    [Test_RelationLabel[docid]],
                                                                                    [Test_ParsingBreaks[docid]],
                                                                                    [Test_GoldenMetric[docid]],
                                                                                    use_pred_segmentation=False,
                                                                                    use_org_Parseval=use_org_Parseval,
                                                                                    batch_size=eval_size,
                                                                                    model=model)
            
            # Unfold numbers
            # Test
            P_span_Test, R_span_Test, F_span_Test = span_points_Test
            P_relation_Test, R_relation_Test, F_relation_Test = relation_points_Test
            P_nuclearity_Test, R_nuclearity_Test, F_nuclearity_Test = nuclearity_points_Test
            P_segment_Test, R_segment_Test, F_segment_Test = segment_points_Test
            
            print('Epoch EvalOnly Test:\t%d\tdocid:\t%d\n' % (best_epoch_Dev, docid),
                  'F_segment_Test\tF_span_Test\tF_nuclearity_Test\tF_relation_Test:\n%.4f\t%.4f\t%.4f\t%.4f'
                  % (F_segment_Test, F_span_Test, F_nuclearity_Test, F_relation_Test))

Epoch EvalOnly Test:	27
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.6804	0.5069	0.4573
Epoch EvalOnly Test:	27	docid:	0
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.7158	0.4947	0.4632
Epoch EvalOnly Test:	27	docid:	1
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.6750	0.5000	0.4500
Epoch EvalOnly Test:	27	docid:	2
 F_segment_Test	F_span_Test	F_nuclearity_Test	F_relation_Test:
1.0000	0.6574	0.5278	0.4630


In [10]:
LoopNeeded = int(np.ceil(len(Test_EDUBreaks) / batch_size))

Loss_tree_all = []
Loss_label_all = []
correct_span = 0
correct_relation = 0
correct_nuclearity = 0
correct_full = 0
no_system = 0
no_golden = 0
no_gold_seg = 0
no_pred_seg = 0
no_correct_seg = 0

correct_span_list = []
correct_relation_list = []
correct_nuclearity_list = []
no_system_list = []
no_golden_list = []

all_label_gold = []
all_label_pred = []

In [11]:
from Training import getBatchData

with torch.no_grad():
    for loop in range(LoopNeeded):

        StartPosition = loop * batch_size
        EndPosition = (loop + 1) * batch_size
        if EndPosition > len(Test_EDUBreaks):
            EndPosition = len(Test_EDUBreaks)

        InputSentences_batch, EDUBreaks_batch, _, RelationLabel_batch, ParsingBreaks_batch, GoldenMetric_batch = \
            getBatchData(Test_InputSentences[StartPosition:EndPosition],
                         Test_EDUBreaks[StartPosition:EndPosition],
                         Test_DecoderInput[StartPosition:EndPosition],
                         Test_RelationLabel[StartPosition:EndPosition],
                         Test_ParsingBreaks[StartPosition:EndPosition],
                         Test_GoldenMetric[StartPosition:EndPosition], batch_size)

        Loss_tree_batch, Loss_label_batch, SPAN_batch, Label_Tuple_batch, predict_EDU_breaks = model.TestingLoss(
            InputSentences_batch, EDUBreaks_batch, RelationLabel_batch,
            ParsingBreaks_batch, GenerateTree=True, use_pred_segmentation=False)

        all_label_gold.extend(Label_Tuple_batch[0])
        all_label_pred.extend(Label_Tuple_batch[1])

        Loss_tree_all.append(Loss_tree_batch)
        Loss_label_all.append(Loss_label_batch)

        correct_span_batch, correct_relation_batch, correct_nuclearity_batch, correct_full_batch, no_system_batch, no_golden_batch, \
        correct_span_batch_list, correct_relation_batch_list, correct_nuclearity_batch_list, \
        no_system_batch_list, no_golden_batch_list, segment_results_list = getBatchMeasure(SPAN_batch,
                                                                                           GoldenMetric_batch,
                                                                                           predict_EDU_breaks,
                                                                                           EDUBreaks_batch,
                                                                                           use_org_Parseval)

        correct_span = correct_span + correct_span_batch
        correct_relation = correct_relation + correct_relation_batch
        correct_nuclearity = correct_nuclearity + correct_nuclearity_batch
        correct_full = correct_full + correct_full_batch
        no_system = no_system + no_system_batch
        no_golden = no_golden + no_golden_batch
        no_gold_seg += segment_results_list[0]
        no_pred_seg += segment_results_list[1]
        no_correct_seg += segment_results_list[2]

        correct_span_list += correct_span_batch_list
        correct_relation_list += correct_relation_batch_list
        correct_nuclearity_list += correct_nuclearity_batch_list
        no_system_list += no_system_batch_list
        no_golden_list += no_golden_batch_list

In [12]:
span_points, relation_points, nuclearity_points, F1_Full, segment_points = getMicroMeasure(correct_span,
                                                                                           correct_relation,
                                                                                           correct_nuclearity,
                                                                                           correct_full,
                                                                                           no_system,
                                                                                           no_golden,
                                                                                           no_gold_seg,
                                                                                           no_pred_seg,
                                                                                           no_correct_seg)

In [13]:
with torch.no_grad():
    Loss_tree, Loss_label, SPAN, Label_Tuple, predict_EDU_breaks = model.TestingLoss(
        Test_InputSentences, Test_EDUBreaks, Test_RelationLabel,
        Test_ParsingBreaks, GenerateTree=True, use_pred_segmentation=False)

In [14]:
correct_span_1, correct_relation_1, correct_nuclearity_1, correct_full_1, no_system_1, no_golden_1, \
        correct_span_list_1, correct_relation_list_1, correct_nuclearity_list_1, \
        no_system_list_1, no_golden_list_1, segment_results_list_1 = getBatchMeasure(SPAN,
                                                                                     Test_GoldenMetric,
                                                                                     predict_EDU_breaks,
                                                                                     Test_EDUBreaks,
                                                                                     use_org_Parseval)

In [15]:
predict_dic_list = []
for i in range(len(SPAN)):
    predict_dic_list.append(Metric.getEvalData_parseval(SPAN[i][0], predict_EDU_breaks[i]))

In [16]:
gold_dic_list = []
for i in range(len(Test_GoldenMetric)):
    gold_dic_list.append(Metric.getEvalData_parseval(Test_GoldenMetric[i][0], Test_EDUBreaks[i]))

# Report on Accuracy

In [17]:
def get_accuracy(predict_dic_list, gold_dic_list):
    
    span_correct = 0
    nuc_correct = 0
    rel_correct = 0
    length = 0
    
    for i in range(len(gold_dic_list)):
        length += len(gold_dic_list[i])
        for key in gold_dic_list[i].keys():
            if key in predict_dic_list[i].keys():
                span_correct += 1
                if gold_dic_list[i][key][0] == predict_dic_list[i][key][0]:
                    rel_correct += 1
                if gold_dic_list[i][key][1] == predict_dic_list[i][key][1]:
                    nuc_correct += 1
    
    span = span_correct / length
    nuc = nuc_correct / length
    rel = rel_correct / length
    
    return span, nuc, rel

In [18]:
span, nuc, rel = get_accuracy(predict_dic_list, gold_dic_list)
print(span)
print(nuc)
print(rel)

0.6804407713498623
0.5068870523415978
0.4573002754820937


# Report on Rel Types

In [19]:
def report_rel_type(predict_dic_list, gold_dic_list):
    
    relname_accuracy = {}
    
    for i in range(len(gold_dic_list)):
        for key in gold_dic_list[i].keys():
            rel = gold_dic_list[i][key][0]
            nuc = gold_dic_list[i][key][1]
            if rel not in relname_accuracy:
                relname_accuracy[rel] = [0, 1]
            else:
                relname_accuracy[rel][1] += 1
            if key in predict_dic_list[i].keys():
                if rel == predict_dic_list[i][key][0] and nuc == predict_dic_list[i][key][1]:
                    relname_accuracy[rel][0] +=1
    
    return relname_accuracy

In [20]:
relname_accuracy = report_rel_type(predict_dic_list, gold_dic_list)

In [21]:
relname_accuracy

{'organization': [4, 6],
 'elaboration': [41, 76],
 'attribution': [1, 6],
 'joint': [54, 112],
 'context': [4, 27],
 'same-unit': [31, 45],
 'adversative': [7, 14],
 'contingency': [0, 7],
 'purpose': [1, 4],
 'explanation': [16, 30],
 'restatement': [0, 1],
 'mode': [4, 10],
 'causal': [2, 20],
 'evaluation': [0, 4],
 'topic': [0, 1]}

# Report on Intra- and Inter-sentential Performances

In [22]:
sent_seg = []
for i in range(len(Test_EDUBreaks)):
    doc_sent_seg = []
    for index in Test_EDUBreaks[i]:
        if Test_InputSentences[i][index] in ['。', '？', '！', '.', '?', '!']:
            doc_sent_seg.append(index)
    sent_seg.append(doc_sent_seg)

In [23]:
def filter_intra(dic_list, sent_seg):
    
    filtered = []
    
    for i in range(len(dic_list)):
        
        result = {}
        
        for key in dic_list[i].keys():
            start, end = map(int, key.split('-'))
            count_in_range = sum(start <= num < end for num in sent_seg[i])
            if count_in_range == 0:
                result.update({key: dic_list[i][key]})
                
        filtered.append(result)
    
    return filtered

In [24]:
intra_gold = filter_intra(gold_dic_list, sent_seg)

In [25]:
inter_gold = []

for i in range(len(gold_dic_list)):
    inter_gold.append({key : value for key, value in gold_dic_list[i].items() if key not in intra_gold[i]})

In [26]:
report_rel_type(predict_dic_list, intra_gold)

{'elaboration': [41, 53],
 'same-unit': [31, 45],
 'joint': [37, 53],
 'contingency': [0, 7],
 'purpose': [1, 4],
 'adversative': [7, 11],
 'mode': [4, 10],
 'explanation': [16, 21],
 'context': [1, 8],
 'causal': [2, 12],
 'attribution': [1, 4]}

In [27]:
report_rel_type(predict_dic_list, inter_gold)

{'organization': [4, 6],
 'attribution': [0, 2],
 'joint': [17, 59],
 'context': [3, 19],
 'adversative': [0, 3],
 'elaboration': [0, 23],
 'explanation': [0, 9],
 'restatement': [0, 1],
 'causal': [0, 8],
 'evaluation': [0, 4],
 'topic': [0, 1]}

In [28]:
span, nuc, rel = get_accuracy(predict_dic_list, intra_gold)
print(span)
print(nuc)
print(rel)

0.8552631578947368
0.6710526315789473
0.6228070175438597


In [29]:
span, nuc, rel = get_accuracy(predict_dic_list, inter_gold)
print(span)
print(nuc)
print(rel)

0.3851851851851852
0.22962962962962963
0.17777777777777778


# Report on Intra-sentential Spans in the Whole Dataset

In [30]:
EDUBreaks = []
GoldenMetric = []
InputSentences = []
EDUBreaks.extend(pickle.load(open(os.path.join(data_path, "EDUBreaks.pickle"), "rb")))
GoldenMetric.extend(pickle.load(open(os.path.join(data_path, "GoldenLabelforMetric.pickle"), "rb")))
InputSentences.extend(pickle.load(open(os.path.join(data_path, "InputSentences.pickle"), "rb")))

In [31]:
seg = []
for i in range(len(EDUBreaks)):
    doc_sent_seg = []
    for index in EDUBreaks[i]:
        if InputSentences[i][index] in ['。', '？', '！', '.', '?', '!']:
            doc_sent_seg.append(index)
    seg.append(doc_sent_seg)

In [32]:
gold_dic_list_all = []
for i in range(len(GoldenMetric)):
    gold_dic_list_all.append(Metric.getEvalData_parseval(GoldenMetric[i][0], EDUBreaks[i]))

In [33]:
intra_all = filter_intra(gold_dic_list_all, seg)

In [34]:
print(sum(len(doc) for doc in intra_all) / sum(len(doc) for doc in gold_dic_list_all))

0.6883755222413369
